In [3]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle as pkl

# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)

# Create training data
X_train, X_test, y_train, y_test = train_test_split(
    data_combined, labels_combined, test_size=.2, random_state=42)

# create the model
class CNN1D(models.Model):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()

        in_channels = 2
        in_features = 128

        # define convolutional layers
        self.conv1 = layers.Conv1D(64, kernel_size=3, padding='same', activation='relu', input_shape=(in_features, in_channels))
        self.maxpool1 = layers.MaxPooling1D(pool_size=2)

        self.conv2 = layers.Conv1D(128, kernel_size=3, padding='same', activation='relu')
        self.maxpool2 = layers.MaxPooling1D(pool_size=2)

        # define the fully connected layers
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(256, activation='relu')
        self.fc2 = layers.Dense(num_classes)

    def call(self, x):
        # input shape: (batch_size, 2, 128)

        # convolutional layers
        x = self.conv1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.maxpool2(x)

        # flatten (before fully connected layers)
        x = self.flatten(x)

        # fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)

        return x

# create the model
model = CNN1D(num_classes=2)

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# reshape the data
X_train = X_train.reshape(-1, 128, 2)
X_test = X_test.reshape(-1, 128, 2)

# train the model
epochs = 10
model.fit(X_train, to_categorical(y_train), epochs=epochs)

# evaluate the model
results = model.evaluate(X_test, to_categorical(y_test))

# print results
print(f'Test Loss: {results[0]} \nTest Accuracy: {results[1]}')


Epoch 1/10
50/50 [==============================] - 0s 4ms/step - loss: 0.8429 - accuracy: 0.5081
Epoch 2/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6935 - accuracy: 0.4925
Epoch 3/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6940 - accuracy: 0.5125
Epoch 4/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.4944
Epoch 5/10
50/50 [==============================] - 0s 4ms/step - loss: 0.6934 - accuracy: 0.5069
Epoch 6/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6941 - accuracy: 0.5181
Epoch 7/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5056
Epoch 8/10
50/50 [==============================] - 0s 4ms/step - loss: 0.6936 - accuracy: 0.5000
Epoch 9/10
50/50 [==============================] - 0s 3ms/step - loss: 0.6937 - accuracy: 0.5100
Epoch 10/10
13/13 [==============================] - 0s 1ms/step - loss: 0.6929 - accuracy: 0.5025
Test Loss: 0.692938